In [3]:
!pip install PyDictionary
!pip install elasticsearch
from PyDictionary import PyDictionary
import json
from nltk.corpus import wordnet
import elasticsearch

     |████████████████████████████████| 327kB 12.4MB/s 


In [4]:
dictionary = PyDictionary()

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
    return list(set(synonyms))


In [5]:
greet = {
    "type": "greet",
    "query": {"data": []},
    "response": "greet_back",
    "starters": ["how are you", "hey", "hey there", "hello", "good morning", "good evening", "hi", "hiie", "hie", "morning"]
}
policy = {
    "type": "policy",
    "query": {"data": []},
    "response": "display_policy",
    "starters": ["policy", "policies", "policies please", "policy please", "show me a list of policies", "I want to see policies", "some policies", "hey can you show me some policies"]
}
ivr = {
    "type": "ivr",
    "query": {"data": []},
    "response": "connect_ivr",
    "starters": ["I would like to speak with an agent", "agent", "I want to speak with someone", "can you connect me with an officer", "can you connect me with someone"]
}
current_balance = {
    "type": "current_balance",
    "query": {"data": []},
    "response": "display_current_balance",
    "starters": ["what is my current balance", "current balance", "balance", "available balance", " remaining balance", "can you show me my balance", "left amount", "display balance please"]
}
bye = {
    "type": "leave",
    "query": {"data": []},
    "response": "greet_goodbye",
    "starters": ["by see you", "have a good day", "see ya", "ok bye", "bye bye"]
}
bike_policy = {
    "type": "bike_policy",
    "query": {"data": []},
    "response": "display_bike_policy",
    "starters": ["I want to see some bike policies", "show me some policies related to two wheeler", "two wheel policies please", " any bike policies", "what about some two wheel policies", "bike policies"]
}
car_policy = {
    "type": "car_policy",
    "query": {"data": []},
    "response": "display_car_policy",
    "starters": ["I want to see some car policies", "show me some policies related to four wheeler", "four wheel policies please", " any car policies", "what about some four wheel policies", "car policies"]
}
health_policy = {
    "type": "medical policy",
    "query": {"data": []},
    "response": "display_medical_policy",
    "starters": ["I want to see some health policies", "show me some policies related to medical", "medical policies please", " any car policies", "what about some medical policies", "health policies"]
}
x_train = [greet, policy, ivr, current_balance, bye, bike_policy, car_policy, health_policy]
final = []


In [6]:
def dataset():
    for intent in x_train:
        print("processing", intent["type"])
        for x in intent["starters"]:
            print("processing", x)
            intent["query"]["data"].append(x)
            tokens = x.split(" ")
            variants = []
            for i, t in enumerate(tokens):
                syno = dictionary.synonym(t)
                if syno:
                    for s in syno:
                        rest = " ".join(tokens[0: i]) + " " + s + " " + " ".join(tokens[(i + 1):len(tokens)])
                        intent["query"]["data"].append(rest)
        final.append(intent)

    file = json.dumps(final)
    with open("dataset.json", "w") as f:
        f.write(file)

In [7]:
def indexig():
    client = elasticsearch.Elasticsearch()
    indexed = 0
    not_indexed = 0
    with open("dataset.json", "r") as f:
        data_json = json.loads(f.read())
        for intent in data_json:
            for message in intent["query"]["data"]:
                if message:
                    query = {
                        "type": intent["type"],
                        "message": message,
                        "response": intent["response"]
                    }
                    try:
                        client.index(index="chatbot", body=query)
                        indexed = indexed + 1
                    except:
                        not_indexed = not_indexed + 1
                else:
                    not_indexed = not_indexed + 1

    print("total indexed", indexed)
    print("total not indexed", not_indexed)

In [8]:
def start_chat():
    client = elasticsearch.Elasticsearch()
    while True:
        message = input(":")
        query = {
            "query": {
                "match": {
                    "message": {
                        "query": message,
                        "fuzziness": "AUTO"
                    }
                }
            }
        }
        ret = client.search(body=query, index="chatbot")
        hits = ret["hits"]["hits"]
        all_results = []
        for h in hits:
            all_results.append(h["_source"]["response"])
        print(json.dumps(all_results))

In [9]:
start_chat()

:Hello


ConnectionError: ignored